In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

In [2]:
# ====== Smooth blednshapes before ratargetting them onto the avatar ======
def smooth_array(arr, window=4):
    kernel = np.array([1] * window) / window
    smoothed_arr = np.ones_like(arr)
    for i in range(arr.shape[1]):
        smoothed_arr[:, i] = np.convolve(arr[:, i], kernel, mode='same')    
    return smoothed_arr

def smooth_and_get_list_of_blendshapes(arkit_list, window=4): 
    video_coeffs = []
    for row in arkit_list:
        frame_coeffs = []
        for category in row:
            if category != 'Timecode' and category != 'BlendshapeCount':
                # prepare for smoothing
                frame_coeffs.append(row[category]) 
        frame_coeffs = np.array(frame_coeffs)
        video_coeffs.append(frame_coeffs)
    video_coeffs = np.stack(video_coeffs, axis=0)
    video_coeffs = smooth_array(video_coeffs, window=window)
    # print(video_coeffs)

    # Put video_coeffs back to the arkit_list
    new_arkit_list = []
    for i, row in enumerate(arkit_list):
        new_row = {}
        j = 0
        for category in row:
            if category != 'Timecode' and category != 'BlendshapeCount':
                new_row[category] = video_coeffs[i, j]
                j += 1
            else:
                new_row[category] = row[category]
        # print(new_row)
        new_arkit_list.append(new_row)

    print(len(new_arkit_list), len(arkit_list))

    print("new_arkit_list[10] == arkit_list[10]: ", new_arkit_list[10] == arkit_list[10])
    return new_arkit_list

In [ ]:
# read existing csv files
path = "predicted_blendshapes"
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith("_blendshapes.csv"):
            
            filename = file.split(".")[0]
            df_blendshapes = pd.read_csv(os.path.join(root, file))
            
            # df_blendshapes to list of dictionaries
            arkit_list = df_blendshapes.to_dict('records')
            # print(len(arkit_list))

            window = 4

            new_arkit_list = smooth_and_get_list_of_blendshapes(arkit_list, window=window)

            arkit_df = pd.DataFrame(new_arkit_list)

            arkit_df.to_csv("predicted_blendshapes/smoothed/" + filename + f"_smooth-{window}.csv", index=False)